# Traduction

L’une des forces motrices qui a permis à la civilisation humaine de se développer est la capacité de communication. Dans la plupart des activités humaines, la communication est essentielle.

![Robot multilingue](./images/translation.jpg)

L’intelligence artificielle (IA) permet de simplifier la communication en traduisant des éléments de texte ou de parole entre des langues, ce qui permet de lever les obstacles à la communication entre les pays et les cultures.

## Créer une ressource Cognitive Services

Dans Azure, vous pouvez utiliser les services cognitifs pour traduire dans plusieurs langues.

Si vous n’avez pas encore de ressources, suivez ces étapes pour créer une ressource **Cognitive Services** dans votre abonnement Azure :

> **Remarque** : Si vous disposez déjà d’une ressource Cognitive Services, il suffit d’ouvrir sa page **Démarrage rapide** dans le portail Azure et de copier sa clé et son point de terminaison dans la cellule ci-dessous. Sinon, suivez les étapes ci-dessous pour en créer une.

1. Sous un autre onglet du navigateur, ouvrez le portail Azure à l’adresse https://portal.azure.com, en vous connectant avec votre compte Microsoft.
2. Cliquez sur le bouton **&#65291; Créer une ressource**, recherchez *Cognitive Services*, puis créez une ressource **Cognitive Services** avec les paramètres suivants :
    - **Abonnement** : *Votre abonnement Azure*.
    - **Groupe de ressources** : *Sélectionnez ou créez un groupe de ressources portant un nom unique*.
    - **Région** : *Choisissez une région disponible* :
    - **Nom** : *Saisissez un nom unique*.
    - **Niveau tarifaire** : S0
    - **Je confirme avoir lu et compris les avis** : Sélectionné.
3. Attendez la fin du déploiement. Ensuite, accédez à votre ressource Cognitive Services et, sur la page **Aperçu**, cliquez sur le lien permettant de gérer les clés du service. Vous aurez besoin du point de terminaison et des clés pour vous connecter à votre ressource Cognitive Services à partir d’applications clientes.

### Obtenir la clé et l’emplacement de votre ressource Cognitive Services

Pour utiliser votre ressource Cognitive Services, les applications clientes ont besoin de sa clé d’authentification et de son emplacement :

1. Dans le portail Azure, sur la page **Clés et Point de terminaison** de votre ressource Cognitive Services, copiez la **Clé 1** de votre ressource et collez-la dans le code ci-dessous, en remplaçant la valeur **YOUR_COG_KEY**.
2. Copiez l’**Emplacement** de votre ressource et collez-le dans le code ci-dessous, en remplaçant la valeur **YOUR_COG_LOCATION**.
>**Remarque** : Restez sur la page **Clés et Point de terminaison** et copiez **l’emplacement** de cette page (par exemple : _westus_). _N’ajoutez_ pas d’espaces entre les mots dans le champ Emplacement. 
3. Exécutez le code ci-dessous en cliquant sur le bouton **Exécuter la cellule** (&#9655;) à gauche de la cellule.

In [ ]:
cog_key = 'YOUR_COG_KEY'
cog_location = 'YOUR_COG_LOCATION'

print('Ready to use cognitive services in {} using key {}'.format(cog_location, cog_key))

## Traduction de texte

Comme son nom l’indique, le service **Traducteur de texte** permet de traduire du texte d’une langue dans une autre.

Il n’existe pas de SDK Python pour ce service, mais vous pouvez utiliser son interface REST pour soumettre des requêtes à un point de terminaison via HTTP. Ceci est relativement facile dans Python, en utilisant la bibliothèque **Requêtes**. Les informations concernant le texte à traduire et le texte traduit résultant sont échangées au format JSON.

Exécutez la cellule suivante pour créer la fonction correspondante, puis testez la fonction avec une simple traduction de l’anglais au français.

In [ ]:
# Create a function that makes a REST request to the Text Translation service
def translate_text(cog_location, cog_key, text, to_lang='fr', from_lang='en'):
    import requests, uuid, json

    # Create the URL for the Text Translator service REST request
    path = 'https://api.cognitive.microsofttranslator.com/translate?api-version=3.0'
    params = '&from={}&to={}'.format(from_lang, to_lang)
    constructed_url = path + params

    # Prepare the request headers with Cognitive Services resource key and region
    headers = {
        'Ocp-Apim-Subscription-Key': cog_key,
        'Ocp-Apim-Subscription-Region':cog_location,
        'Content-type': 'application/json',
        'X-ClientTraceId': str(uuid.uuid4())
    }

    # Add the text to be translated to the body
    body = [{
        'text': text
    }]

    # Get the translation
    request = requests.post(constructed_url, headers=headers, json=body)
    response = request.json()
    return response[0]["translations"][0]["text"]


# Test the function
text_to_translate = "Hello"

translation = translate_text(cog_location, cog_key, text_to_translate, to_lang='fr', from_lang='en')
print('{} -> {}'.format(text_to_translate,translation))

Le service doit avoir traduit le texte anglais « Hello » en français « Bonjour ».

Notez que les langues sont spécifiées en utilisant un système standard d’abréviations de langue, avec *en* pour l’anglais et *fr* pour le français. Vous pouvez également utiliser des abréviations qui incluent des cultures spécifiques, ce qui est utile lorsqu’une même langue est utilisée dans différentes zones géographiques, souvent avec des orthographes différentes. Par exemple, *en-US* désigne l’anglais des États-Unis, tandis que *en-GB* indique l’anglais de Grande-Bretagne.

Exécutez la cellule suivante pour traduire l’anglais britannique en italien.

In [ ]:
text_to_translate = "Hello"

translation = translate_text(cog_location, cog_key, text_to_translate, to_lang='it-IT', from_lang='en-GB')
print('{} -> {}'.format(text_to_translate,translation))

Essayons une autre traduction, cette fois de l’anglais américain au chinois.

In [ ]:
text_to_translate = "Hello"

translation = translate_text(cog_location, cog_key, text_to_translate, to_lang='zh-CN', from_lang='en-US')
print('{} -> {}'.format(text_to_translate,translation))

## Traduction vocale

Vous pouvez utiliser le service **Parole** pour traduire le langage parlé.

Vous pouvez maintenant exécuter la cellule suivante pour créer et tester une fonction qui utilise le SDK Parole pour traduire la parole audible.

> **Remarque** : Vous aurez besoin de haut-parleurs pour entendre l’audio.

In [ ]:
from playsound import playsound 

# Create a function to translate audio in one language to text in another
def translate_speech(cog_location, cog_key, audio_file=None, to_lang='fr-FR', from_lang='en-US'):
    from azure.cognitiveservices.speech import SpeechConfig, AudioConfig, ResultReason
    from azure.cognitiveservices.speech.translation import SpeechTranslationConfig, TranslationRecognizer

    # Configure the speech translation service
    translation_config = SpeechTranslationConfig(subscription=cog_key, region=cog_location)
    translation_config.speech_recognition_language = from_lang
    translation_config.add_target_language(to_lang)

    # Configure audio input
    if audio_file is None:
        audio_config = AudioConfig() # Use default input (microphone)
    else:
        audio_config = AudioConfig(filename=audio_file) # Use file input

    # Create a translation recognizer and use it to translate speech input
    recognizer = TranslationRecognizer(translation_config, audio_config)
    result = recognizer.recognize_once()

    # Did we get it?
    translation = ''
    speech_text = ''
    if result.reason == ResultReason.TranslatedSpeech:
        speech_text = result.text
        translation =  result.translations[to_lang]
    elif result.reason == ResultReason.RecognizedSpeech:
        speech_text = result.text
        translation =  'Unable to translate speech'
    else:
        translation = 'Unknown'
        speech_text = 'Unknown'

    # return the translation
    return speech_text, translation
    

# Test the function
import os

file_name = 'english.wav'
file_path = os.path.join('data', 'translation', file_name)
speech, translated_speech = translate_speech(cog_location, cog_key, file_path, to_lang='es', from_lang='en-US')
result = '{} -> {}'.format(speech, translated_speech)

# Show translated text
playsound(file_path)
print(result)

Notez que la langue « cible » doit être identifiée à l’aide d’un code de langue à 2 caractères (par exemple *en*), tandis que la langue « source » doit inclure l’indicateur de culture (par exemple *en-US*).

Essayons de traduire du français vers l’anglais.

In [ ]:
from playsound import playsound
import os

file_name = 'french.wav'
file_path = os.path.join('data', 'translation', file_name)
speech, translated_speech = translate_speech(cog_location, cog_key, file_path, to_lang='en', from_lang='fr-FR')
result = '{} -> {}'.format(speech, translated_speech)

# Show translated text
playsound(file_path)
print(result)

## En savoir plus

Vous pouvez en savoir plus sur le [Traducteur de texte](https://docs.microsoft.com/azure/cognitive-services/translator/) et sur la [traduction avec le service Parole](https://docs.microsoft.com/azure/cognitive-services/speech-service/index-speech-translation) dans la documentation du service.